# Lab 8

## Crawling Wikipedia
Below, you'll find a Python program. Feel free to run it. The function `find_links` extracts URLs from a web page. The functions `bfs` and `dfs` perform breadth-first and depth-first searches of the links on a web page. Change the parameter `start_url` (it's waaaay down at the bottom of the program) to the Wikipedia entry for your favorite topic. Pick something you know a lot about. Run the program. It should print out the entries visited along the way. Try different values for `max_visits` and `start_url`. Also, try replacing bfs with dfs. Note that algorithm randomly chooses which links to visit, so you can run the program with the same inputs and see different outputs.

In [ ]:
'''
Initialization stuff. Don't worry about what it does
'''
from bs4 import BeautifulSoup
import re, urllib, random
p=re.compile("\/wiki\/([^:]+)")
ignore=set(["Wikipedia","Help","Portal","Category","Main_Page","File","Talk","Template","Template_talk","Special"])

'''
Function to extract all of the URLs from a Wikipedia page
'''
def get_links(page):
    page="https://en.wikipedia.org/wiki/%s"%page
    r = urllib.request.urlopen(page).read()
    soup = BeautifulSoup(r,'html')

    links=[]
    for tag in soup.find_all('a'):
        s=tag.get('href')
        if s:
            m=p.match(s)
            if m:
                newurl=m.group(1)
                if newurl not in ignore:
                    #newurl=urllib.quote(newurl.encode('utf8'), ':/')
                    links.append(newurl)
    random.shuffle(links)
    return links

'''
A depth first search function that starts at url and visits max_visits nodes
'''
def dfs(url,max_visits):
    visited=[]
    stack=[]
    stack.append(url)
    while stack:
        if len(visited)>=max_visits:
            return visited
        current=stack.pop()
        if current not in visited:
            for neighbor in get_links(current):
                stack.append(neighbor)
            visited.append(current)


'''
A breadth first search function that starts at url and visits max_visits nodes
'''
def bfs(url,max_visits):
    visited=[]
    stack=[]
    stack.append(url)
    while stack:
        if len(visited)>=max_visits:
            return visited
        current=stack.pop(0)
        if current not in visited:
            for neighbor in get_links(current):
                stack.append(neighbor)
            visited.append(current)
            
'''
Main program for you to play with
'''
start_url="Alan_Turing"     # the starting url for our crawl
max_visits=50               # the max number of nodes to visit
for i,link in enumerate(bfs(start_url,max_visits)):
    print(i,link)

## First Trie
In this section, we will look at another application for graphs we didn’t discuss in class. Feel free to run the program below. You can change the variable sentence to anything you like. Try running the program a few times with different sentences. The program produces directed graphs, but they look a little different than what we saw in class. Thick lines are used instead of arrows to indicate the direction of the edge. If the graph visualization is difficult to see, note that the program also saves the image as a file named `trie.png`.

In [ ]:
END="_end" # special string representing the end of a word

'''
adds a word to our trie
'''
def add_word(root,word):
    current=root
    for c in word:
        if c in current:
            current=current[c]
        else:
            current[c]={}
            current=current[c]
    if END in current:
        current[END]+=1
    else:
        current[END]=1


def draw_graph(root):
    import networkx as nx
    from networkx.drawing.nx_agraph import write_dot, graphviz_layout
    import matplotlib.pyplot as plt

    G=nx.DiGraph()
    node_names={"":1}
    G.add_node(1)
    dfs(root,"",G,node_names)
    plt.rc('figure', figsize=(20,20))
    pos = nx.shell_layout(G)
    nx.draw(G,pos=pos,node_color="lightsteelblue")
    nx.draw_networkx_labels(G,pos)
    nx.draw_networkx_edge_labels(G,pos,edge_labels=nx.get_edge_attributes(G,'label'))
    plt.savefig("trie.png", format="PNG")
    
def dfs(node,s,G,node_names):
    for c in node:
        if c != END:
            sc=s+c
            if sc not in node_names:
                node_names[sc]=len(node_names)+1
                G.add_node(node_names[sc])
            G.add_edge(node_names[s],node_names[sc],label=c)
            dfs(node[c],sc,G,node_names)

root={}
sentence="she sells sea shells down by the sea shore"
for word in sentence.split():
    add_word(root,word)

draw_graph(root)

## Trie Trie Again
In this section, we’ll look at tries again. The program here is almost exactly the same as in the previous section. Instead of visualizing the trie, we’re going to examine the `find_word` function. Experiment with different values for sentence and query.

In [ ]:
END="_end" # special string representing the end of a word

'''
adds a word to our trie
'''
def add_word(root,word):
    current=root
    for c in word:
        if c in current:
            current=current[c]
        else:
            current[c]={}
            current=current[c]
    if END in current:
        current[END]+=1
    else:
        current[END]=1
        
'''
finds a word in our trie
'''
def find_word(root,word):
    current=root
    for c in word:
        if c in current:
            current=current[c]
        else:
            return 0
    if END in current:
        return current[END]
    else:
        return 0   

root={}
sentence="she sells sea shells down by the sea shore"
query="sea"

for word in sentence.split():
    add_word(root,word)

print(find_word(root,query))

## Trie Hard
Once again, we are going to look at tries. Instead of a sentence you supply, this program will load all of the words in the classic book Frankenstein. The *entire contents* of the novel are stored in `frankenstein.txt`. Again, you should try examining different values for `query`. This is the word we're searching for in the book. The program will tell you how long it took to read and process the book, how long it took to search the book for the word, and how many times that word appeared in the book. I hope you're impressed!

In [ ]:
from timeit import default_timer as timer
END="_end" # special string representing the end of a word

# adds a word to our trie
def add_word(root,word):
    current=root
    for c in word:
        if c in current:
            current=current[c]
        else:
            current[c]={}
            current=current[c]
    if END in current:
        current[END]+=1
    else:
        current[END]=1
        
# finds a word in our trie
def find_word(root,word):
    current=root
    for c in word:
        if c in current:
            current=current[c]
        else:
            return 0
    if END in current:
        return current[END]
    else:
        return 0
    
    
def read_book():
    import re
    f=open("frankenstein.txt",'r')
    text=f.read()
    f.close()
    words= re.split("[^a-zA-Z0-9]",text)
    return [w.lower() for w in words if len(w)>2 ]

start=timer()
words=read_book()
end=timer()
time=end-start
print("Reading the book took %s seconds." % time)

root={}
for word in words:
    add_word(root,word)

query="monster"
start=timer()
result=find_word(root,query)
end=timer()
time=end-start
print("Searching the book took %s seconds." % time)
print("The result of the search was: %d" % result)

## Lab Questions

1. In the crawling program what role does `max_visits` play? Why do we need this parameter?
2. In the crawling program, how does the output from `dfs` differ from `bfs`? How would you describe what you see to a friend who hasn’t taken this class? Is there an intuitive difference between the way DFS and BFS explore topics?
3. Are the searches in our crawling program over a tree? In other words, is Wikipedia a tree? Why or why not?
4. Are the graphs produced in "First Trie" a tree? Why or why not?
5. What information is being captured in the graph in "First Trie"? Why might it be useful?
6. What would a depth first search of the graph in "First Trie produce?
7. What is the `find_word` function in "Trie Trie Again" doing? What value does it return?
8. How long does it take to read the book in "Trie Hard"? How long does it take to search the book? Do these results surprise or impress you?
9. In "Trie Hard", does the length of the query string impact the amount of time required to search the book?